In [ ]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# import model_inference


ModuleNotFoundError: No module named 'model_inference'

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 21:46:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/08 21:46:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Config


In [4]:
snapshot_date_str = "2024-01-01"
model_name = "xgb_credit_model_2024_09_01.pkl"


In [5]:
config = {}
config["snapshot_date_str"] = snapshot_date_str
config["snapshot_date"] = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
config["model_name"] = model_name
config["model_bank_directory"] = "model_bank/"
config["model_artefact_filepath"] = config["model_bank_directory"] + config["model_name"]

pprint.pprint(config)

{'model_artefact_filepath': 'model_bank/xgb_credit_model_2024_09_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'xgb_credit_model_2024_09_01.pkl',
 'snapshot_date': datetime.datetime(2024, 1, 1, 0, 0),
 'snapshot_date_str': '2024-01-01'}


In [6]:
# Load the model from the pickle file
with open(config["model_artefact_filepath"], 'rb') as file:
    model_artefact = pickle.load(file)

print("Model loaded successfully! " + config["model_artefact_filepath"])

Model loaded successfully! model_bank/xgb_credit_model_2024_09_01.pkl


In [ ]:
gold_feature_directory = "/app/datamart/gold/feature_store/"

# Read all CSV files into a single DataFrame
files_list = [gold_feature_directory+os.path.basename(f) for f in glob.glob(os.path.join(gold_feature_directory, '*'))]
features_store_sdf = spark.read.option("header", "true").parquet(*files_list)


# extract feature store
features_sdf = features_store_sdf.filter((col("snapshot_date") == config["snapshot_date"]))
print("extracted features_sdf", features_sdf.count(), config["snapshot_date"])

features_pdf = features_sdf.toPandas()
features_pdf

extracted features_sdf 8974 2024-01-01 00:00:00


,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date
0,239,140,-24,265,2,-32,147,-38,280,210,...,235,140,45,179,61,38,68,171,CUS_0x1037,2024-01-01
1,-15,137,35,-9,124,23,-33,-66,134,118,...,305,52,30,214,-125,176,128,209,CUS_0x1069,2024-01-01
2,361,58,-37,150,17,40,189,-64,119,32,...,168,-32,-15,34,162,94,-55,103,CUS_0x114a,2024-01-01
3,96,231,-41,20,147,217,158,-45,8,300,...,249,-4,259,190,97,163,-40,3,CUS_0x1184,2024-01-01
4,54,294,-89,-43,288,355,57,-33,157,216,...,311,191,0,-19,-51,80,69,-26,CUS_0x1297,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,141,154,89,149,155,114,-94,277,93,195,...,147,107,164,176,105,92,126,-19,CUS_0xdf6,2024-01-01
8970,96,52,242,123,207,155,147,189,121,291,...,60,218,132,104,-36,164,-45,158,CUS_0xe23,2024-01-01
8971,75,-7,-42,60,57,237,12,90,225,9,...,90,117,251,153,36,-117,244,3,CUS_0xe4e,2024-01-01
8972,138,153,38,306,282,171,-23,102,281,141,...,-75,43,64,182,210,-73,13,7,CUS_0xedd,2024-01-01
